# Lab 2: Debug a Broken Vanilla GAN (find 12+ issues)

In [1]:
import torch, torchvision, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])  # BUG normalize -donee
loader=DataLoader(torchvision.datasets.MNIST('./data',True,download=True,transform=transform),batch_size=256,shuffle=True,num_workers=2,pin_memory=True)
z_dim=100; g_lr=2e-4; d_lr=2e-4
class D(nn.Module):
    def __init__(self):
        super().__init__(); self.net=nn.Sequential(nn.Conv2d(1,32,4,2,1),nn.LeakyReLU(0.2,True),nn.Conv2d(32,64,4,2,1),nn.BatchNorm2d(64),nn.LeakyReLU(0.2,True),nn.Conv2d(64,1,7,1,0))
    def forward(self,x):return self.net(x).view(x.size(0),1)
class G(nn.Module):
    def __init__(self):
        super().__init__(); self.net=nn.Sequential(nn.ConvTranspose2d(z_dim,128,7,1,0),nn.BatchNorm2d(128),nn.ReLU(True),nn.ConvTranspose2d(128,64,4,2,1),nn.BatchNorm2d(64),nn.ReLU(True),nn.ConvTranspose2d(64,1,4,2,1), nn.Tanh())  # BUG missing Tanh --donne
    def forward(self,z): z=z.view(z.size(0),z_dim,1,1); return self.net(z)  # BUG reshape - donnee
Dnet=D().to(device); Gnet=G().to(device)
crit=nn.BCEWithLogitsLoss(); opt_d=torch.optim.Adam(Dnet.parameters(),lr=d_lr,betas=(0.5,0.999)); opt_g=torch.optim.Adam(Gnet.parameters(),lr=g_lr,betas=(0.5,0.999))

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 484kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.47MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.43MB/s]


In [2]:
from tqdm import tqdm
for real,_ in tqdm(loader):
    real=real.to(device); b=real.size(0)
    opt_d.zero_grad()

    z=torch.randn(b,z_dim,device=device)
    fake=Gnet(z.view(b,z_dim,1,1)).detach()

    loss_d_real = crit(Dnet(real), torch.ones(b, 1, device=device))
    loss_d_fake = crit(Dnet(fake), torch.zeros(b, 1, device=device))
    loss_d = loss_d_real + loss_d_fake

    loss_d.backward()
    opt_d.step()

    opt_g.zero_grad()
    z = torch.randn(b, z_dim, device=device)
    fake = Gnet(z.view(b, z_dim, 1, 1))

    loss_g = crit(Dnet(fake), torch.ones(b, 1, device=device))

    loss_g.backward()
    opt_g.step()
print('Now fix all the issues.')

100%|██████████| 235/235 [00:14<00:00, 16.57it/s]

Now fix all the issues.
